In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import scipy.io as spio
import math
import time
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Conv2D, Dropout, MaxPool1D, Flatten, Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import MeanSquaredError
import matplotlib.pyplot as plt

start_time = time.time()

#hyper parameters
batch_size = 64
epochs = 100
num_classes = 2

SNRs = ["-2", "0", "3", "10", "15", "23", "50"] 
%cd /content/drive/My Drive/AWGN\ Only2/
for snr in SNRs: 
    folder = "SNR"+snr
    #import data
    %cd $folder
    

    '''
    Data shape should be (#, input dim)
    Rows should be greater than columns
    use np.transpose if not formatted correctly
    '''
    matname = "Data_SNR" + snr + ".mat"
    mat = spio.loadmat(matname, squeeze_me=True)
    x_train = mat['x_train']
    #x_train = np.transpose(x_train)
    x_valid = mat['data_val']
    #x_valid = np.transpose(x_valid)
    x_test = mat['x_pred']
    #x_test = np.transpose(x_test)
    y_train = mat['y_train']
    #y_train = np.transpose(y_train)
    y_valid = mat['target_val']
    #y_valid = np.transpose(y_valid)

    %cd ../

    # Convert the data to floats between 0 and 1.
    x_train = x_train.astype('float32')
    x_valid = x_valid.astype('float32')
    x_test = x_test.astype('float32')

    print(x_train.shape, 'train samples')
    print(x_valid.shape, 'valid samples')
    print(y_train.shape, 'train labels')
    print(y_valid.shape, 'valid labels')


    #define the network 
    print('Build model...')



    # Define the network
    model = Sequential()
    model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(18,1)))
   
    model.add(Flatten())
    model.add(Dense(50, activation='tanh'))
    model.add(Dense(num_classes, activation='linear'))



    model.summary()

    #complie model
    model.compile(loss=keras.metrics.mean_squared_error,
                    optimizer=SGD(),
                    metrics=[keras.metrics.RootMeanSquaredError(name='rmse')])
'''

    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=2,
                        validation_data=(x_valid, y_valid))

    #make predictions
    predictions = model.predict(x_test)
    matname = "predictionsSNR" + snr + ".mat"
    spio.savemat(matname, {'pred': predictions})
    !cp $matname /content/drive/MyDrive/NN\ Output
   
    savename = "deep_model_SNR" + snr + ".h5"
    model.save(savename)
    !cp $savename /content/drive/MyDrive/NN\ Output

    elapsedTime = (time.time() - start_time)/60
    print("--- %.2f minutes ---" % elapsedTime)
    
'''


Debug, ignore

In [18]:
 #make predictions
predictions = model.predict(x_test)
matname = "predictionsSNR" + snr + ".mat"
spio.savemat(matname, {'pred': predictions})
!cp $matname /content/drive/MyDrive/NN\ Output

savename = "deep_model_SNR" + snr + ".h5"
model.save(savename)
!cp $savename /content/drive/MyDrive/NN\ Output

In [ ]:
 model.add(MaxPool1D(pool_size=2, strides=None, padding='valid',
    data_format='channels_last'))
    model.add(Conv1D(64, kernel_size=5, activation='relu'))
    model.add(MaxPool1D(pool_size=2, strides=None, padding='valid',
    data_format='channels_last'))
    model.add(Conv1D(64, kernel_size=6, activation='relu'))